## Tests of dataset method

In [1]:
import pandas as pd
from MDRMF import Dataset

from rdkit import Chem
from rdkit.Chem import AllChem
from collections import defaultdict
from rdkit.Chem import rdFMCS
from rdkit.Chem import Draw
from rdkit import DataStructs
from rdkit.Chem import Descriptors

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor

2024-01-18 08:53:00,504	INFO worker.py:1724 -- Started a local Ray instance.


## Prepare Data

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/jensengroup/GB_GA/master/ZINC_250k.smi', header=None)

def mol2fp(mol):
    fp = AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=1024)
    arr = np.zeros((0,))
    DataStructs.ConvertToNumpyArray(fp, arr)
    return arr.astype(int)

df.columns = ['smiles']

df = df.head(100)
df['logP'] = [Descriptors.MolLogP(Chem.MolFromSmiles(s)) for s in df['smiles']]

df_train = df.head(10)
df_train['smiles'] = df_train['smiles'].apply(lambda x: mol2fp(Chem.MolFromSmiles(x)))

X = df_train['smiles']
y = df_train['logP']

dataset = Dataset(X, y, df_train.index)
pairwise_dataset = dataset.create_pairwise_dataset()

# rfr = RandomForestRegressor()
# rfr.fit(dataset.X, dataset.y)

rfr_pairwise = RandomForestRegressor()
rfr_pairwise.fit(pairwise_dataset.X, pairwise_dataset.y)

URLError: <urlopen error [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond>

## Predict

##### Single mol

In [ ]:
mol = 11
s = df['smiles'].iloc[mol]
X_test = []
for st in df_train['smiles']:
    X = list(st) + list(mol2fp(Chem.MolFromSmiles(s))) + (list(st - mol2fp(Chem.MolFromSmiles(s))))
    X_test.append(X)

dy_preds = rfr_pairwise.predict(np.array(X_test))

y_preds = df_train['logP'] + dy_preds
y_pred = y_preds.mean()

print(y_pred, df['logP'].iloc[mol])

3.7420618800000027 3.4174200000000017


##### Multiple mols

In [ ]:
mols_to_predict = slice(11, 20)
smiles_to_predict = df['smiles'].iloc[mols_to_predict]

c = 11
for s in smiles_to_predict:
    X_test = []
    for st in df_train['smiles']:
        X = list(st) + list(mol2fp(Chem.MolFromSmiles(s))) + (list(st - mol2fp(Chem.MolFromSmiles(s))))
        X_test.append(X)

    dy_preds = rfr_pairwise.predict(np.array(X_test))
    y_preds = df_train['logP'] + dy_preds
    y_pred = y_preds.mean()

    print(c, "\t", y_pred, "\t", df['logP'].iloc[c])
    c += 1

11 	 3.7420618800000027 	 3.4174200000000017
12 	 3.066561840000002 	 2.3794000000000004
13 	 3.1616786000000014 	 3.6157200000000023
14 	 3.769888960000003 	 1.8117999999999994
15 	 3.5134329000000024 	 0.6130000000000004
16 	 3.0337255600000015 	 4.2250400000000035
17 	 3.1831646400000024 	 2.1622000000000003
18 	 2.6053059800000016 	 3.3920200000000023
19 	 2.5616665000000016 	 3.803600000000002


With these results I confirmed that I got the same results as Jan did in his notebook

## Using a dataset with rdkit2D(200 compounds) and docking scores

In [ ]:
dataset_rdkit2d = Dataset.load('datasets/dataset_rdkit2d_200.pkl')

train_points = [x for x in range(10)] # get the first 10 points
test_points = [x for x in range(10, 20)] # get the next 10 points

# Create training set
dataset_rdkit2d_train = dataset_rdkit2d.get_samples(10)

# Create test set
dataset_rdkit2d_test = dataset_rdkit2d.get_samples(10)

# Create pairwise set
dataset_rdkit2d_pairwise = dataset_rdkit2d_train.create_pairwise_dataset()

# Train model
rfr_pairwise_rdkit2d = RandomForestRegressor()
rfr_pairwise_rdkit2d.fit(dataset_rdkit2d_pairwise.X, dataset_rdkit2d_pairwise.y)

RandomForestRegressor()

In [ ]:
def predict(train_dataset: Dataset, predict_dataset: Dataset, model: RandomForestRegressor):

    mols_predict = predict_dataset.X
    mols_train = train_dataset.X
    y_train = train_dataset.y

    preds = []
    for pmol in mols_predict:
        X_test = []
        for tmol in mols_train:
            X = list(tmol) + list(pmol) + (list(tmol - pmol))
            X_test.append(X)      
        
        dy_preds = model.predict(np.array(X_test))
        y_preds = y_train + dy_preds
        y_pred = y_preds.mean()
        preds.append(y_pred)

    return preds

preds = predict(dataset_rdkit2d_train, dataset_rdkit2d_test, rfr_pairwise_rdkit2d)
trues = dataset_rdkit2d_test.y

print('preds', '\t', 'trues')
diffs = []
for i, j in zip(preds, trues):
    print(round(i, 1), '\t', round(j, 1))
    diff = abs(i-j)
    diffs.append(diff)
print('Avg. diff:', np.mean(diffs))

preds 	 trues
-7.5 	 -6.7
-8.1 	 -6.7
-7.5 	 -6.6
-7.9 	 -6.8
-6.0 	 -12.5
-7.8 	 -3.7
-7.2 	 -7.0
-7.3 	 -8.8
-7.9 	 -7.7
-7.2 	 -10.0
Avg. diff: 1.946613488


##### Trying with morgan(512 bits) featurized dataset

In [ ]:
dataset = Dataset.load('datasets/dataset_morgan.pkl')
dataset_morgan_train = dataset.get_samples(10)
dataset_morgan_test = dataset.get_samples(10)

dataset_morgan_pairwise = dataset_morgan_train.create_pairwise_dataset()
rfr_pairwise_morgan = RandomForestRegressor()
rfr_pairwise_morgan.fit(dataset_morgan_pairwise.X, dataset_morgan_pairwise.y)

preds = predict(dataset_morgan_train, dataset_morgan_test, rfr_pairwise_morgan)
trues = dataset_morgan_test.y

print('preds', '\t', 'trues')
diffs = []
for i, j in zip(preds, trues):
    print(round(i, 1), '\t', round(j, 1))
    diff = abs(i-j)
    diffs.append(diff)
print('Avg. diff:', np.mean(diffs))

preds 	 trues
-6.5 	 -7.8
-7.5 	 -4.7
-6.9 	 -8.5
-6.8 	 -9.5
-6.8 	 -4.5
-6.4 	 -8.5
-6.9 	 -5.3
-6.9 	 -7.9
-5.8 	 -6.8
-6.5 	 -10.2
Avg. diff: 2.0295933770000003


In [ ]:
a = 150 ** 2
b = 50 ** 2
c = a/b
print(f'''
It took 25.6 seconds to compute and fit the model with 50 mols in the training set.
For 50 mols there is a 50^2 = {50 ** 2} combinations to be computed, fitted and predicted.
For 150 mols there is 150^2 = {150 ** 2} combinations to be computed, fitted and predicted.
This is a {c} fold increase meaning it should take {round(25.6*9/60, 1)} minutes to compute.
In reality, however, it took 7.21 minutes, because fitting the model and predicting also takes up time.
Additionally, and this probably makes up for the largest delta between the approx. 4 minutes and 7 minutes, is the exponential
amount of combinations to be computed. Finally, memory shortage will become a problem for large datasets.

In conclusion, using pairwise methods is heavy on computation and is very memory intensive for large datasets.
''')


It took 25.6 seconds to compute and fit the model with 50 mols in the training set.
For 50 mols there is a 50^2 = 2500 combinations to be computed, fitted and predicted.
For 150 mols there is 150^2 = 22500 combinations to be computed, fitted and predicted.
This is a 9.0 fold increase meaning it should take 3.8 minutes to compute.
In reality, however, it took 7.21 minutes, because fitting the model and predicting also takes up time.
Additionally, and this probably makes up for the largest delta between the approx. 4 minutes and 7 minutes, is the exponential
amount of combinations to be computed. Finally, memory shortage will become a problem for large datasets.

In conclusion, using pairwise methods is heavy on computation and is very memory intensive for large datasets.



##### Trying with CDDD

In [ ]:
dataset = Dataset.load('datasets/dataset_CDDD.pkl')
dataset_CDDD_train = dataset.get_samples(15)
dataset_CDDD_test = dataset.get_samples(10)

dataset_CDDD_pairwise = dataset_CDDD_train.create_pairwise_dataset()
rfr_pairwise_CDDD = RandomForestRegressor()
rfr_pairwise_CDDD.fit(dataset_CDDD_pairwise.X, dataset_CDDD_pairwise.y)

preds = predict(dataset_CDDD_train, dataset_CDDD_test, rfr_pairwise_CDDD)
trues = dataset_CDDD_test.y

print('preds', '\t', 'trues')
diffs = []
for i, j in zip(preds, trues):
    print(round(i, 1), '\t', round(j, 1))
    diff = abs(i-j)
    diffs.append(diff)
print('Avg. diff:', np.mean(diffs))
    

preds 	 trues
-6.6 	 -9.0
-7.0 	 -7.8
-7.8 	 -9.6
-9.1 	 -4.8
-9.3 	 -8.6
-9.5 	 -7.0
-9.2 	 -7.7
-10.0 	 -7.0
-7.1 	 -8.1
-9.4 	 -6.1
Avg. diff: 2.1331192026666663


## Final test
##### Does the predict function work as expected compared to the original code?

In [ ]:
# First create a dataset from the initial df as used in the beginning of the notebook.
# Featurize the smiles and assign them to X.

X_final = df['smiles'].apply(lambda x: mol2fp(Chem.MolFromSmiles(x)))
print(X_final)
y_final = df['logP']
print(y_final)

0     [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...
1     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
2     [0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
3     [0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
4     [0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
                            ...                        
95    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...
96    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
97    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
98    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...
99    [0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
Name: smiles, Length: 100, dtype: object
0     5.05060
1     3.11370
2     4.96778
3     4.00022
4     3.60956
       ...   
95    4.47690
96    2.57252
97    0.12710
98    1.93192
99    4.62630
Name: logP, Length: 100, dtype: float64


In [ ]:
# Putting it all together in a dataset
dataset_final = Dataset(X=X_final, y=y_final)

# Creating training and test sets
dataset_final_train = dataset_final.get_points([x for x in range(10)]) # First 10 points
dataset_final_test = dataset_final.get_points([x for x in range (11,20)]) # Next 10 points

# Creating pairwise dataset
dataset_final_pairwise = dataset_final_train.create_pairwise_dataset()

# Create and train the model
rfr_pairwise_final = RandomForestRegressor()
rfr_pairwise_final.fit(dataset_final_pairwise.X, dataset_final_pairwise.y)

preds = predict(dataset_final_train, dataset_final_test, rfr_pairwise_final)
trues = dataset_final_test.y

print('preds', '\t', 'trues')
diffs = []
for i, j in zip(preds, trues):
    print(round(i, 1), '\t', round(j, 1))
    diff = abs(i-j)
    diffs.append(diff)
print('Avg. diff:', np.mean(diffs))

KeyboardInterrupt: 

```python
Final code      Initial code

preds 	 trues  preds                   trues
3.7 	 3.4    3.836891180000002 	    3.4174200000000017
2.8 	 2.4    2.9589798800000016 	    2.3794000000000004
3.0 	 3.6    3.017613160000002 	    3.6157200000000023
3.7 	 1.8    3.828545380000002 	    1.8117999999999994
3.5 	 0.6    3.675127240000002 	    0.6130000000000004
3.0 	 4.2    2.9495092000000014 	    4.2250400000000035
3.0 	 2.2    2.9715343200000017 	    2.1622000000000003
2.5 	 3.4    2.536603820000001 	    3.3920200000000023
2.7 	 3.8    2.7803263000000014 	    3.803600000000002
```
### Conclusion
The predict function work as expected
Small variation occurs due to randomness in RFR

## Testing the predict function on known data

In [ ]:
from MDRMF.models import RFModellerPairwise

rfmodellerpairwise = RFModellerPairwise(dataset_final_train)

rfmodellerpairwise._pairwise_predict(dataset_final_train, dataset_final_test, rfr_pairwise_final)

array([2.18887004, 2.1295359 , 2.35322896, 4.47035888, 1.88803395,
       2.09269344, 4.28154531, 1.67515779, 1.74760799])

## Rebuilding algorithm according to article pseudo-code
https://pubs.acs.org/doi/suppl/10.1021/acs.jcim.1c00670/suppl_file/ci1c00670_si_001.pdf 

In [ ]:
import numpy as np
from numpy import concatenate, newaxis, repeat


def PADRE_features(X1, X2):
    n1 = X1.shape[0]
    n2 = X2.shape[0]

    X1 = X1[:, newaxis, :].repeat(n2, axis=1)
    X2 = X2[newaxis, :, :].repeat(n1, axis=0)

    X1X2_combined = concatenate([X1, X2, X1 - X2], axis=2)
    return X1X2_combined.reshape(n1 * n2, -1)

def PADRE_labels(y1, y2):
    return (y1[:, newaxis] - y2[newaxis, :]).flatten()

def PADRE_train(model, train_X, train_y):
    X1X2 = PADRE_features(train_X, train_X)
    y1_minus_y2 = PADRE_labels(train_y, train_y)
    model.fit(X1X2, y1_minus_y2)
    return model

def PADRE_predict(model, test_X, train_X, train_y):
    n1 = test_X.shape[0]
    n2 = train_X.shape[0]

    X1X2 = PADRE_features(test_X, train_X)
    y1_minus_y2_hat = model.predict(X1X2)
    y1_hat_distribution = y1_minus_y2_hat.reshape(n1, n2) + train_y[newaxis, :]
    mu = y1_hat_distribution.mean(axis=1)
    std = y1_hat_distribution.std(axis=1)
    return mu, std

def predict(train_dataset: Dataset, predict_dataset: Dataset, model: RandomForestRegressor):

    mols_predict = predict_dataset.X
    mols_train = train_dataset.X
    y_train = train_dataset.y

    preds = []
    for pmol in mols_predict:
        X_test = []
        for tmol in mols_train:
            X = list(tmol) + list(pmol) + (list(tmol - pmol))
            X_test.append(X)      
        
        dy_preds = model.predict(np.array(X_test))
        y_preds = y_train + dy_preds
        y_pred = y_preds.mean()
        preds.append(y_pred)

    return preds

train_feat_vector1 = np.array([0, 1, 1, 1, 0, 0, 1])
train_feat_vector2 = np.array([1, 1, 1, 0, 1, 0, 1])
train_feat_vector3 = np.array([1, 0, 0, 1, 1, 0, 0])
train_features = np.array([train_feat_vector1, train_feat_vector2, train_feat_vector3])
train_labels = np.array([3, 2, 5])

PADRE_model = RandomForestRegressor()
PADRE_train(PADRE_model, train_features, train_labels)

test_feat_vector1 = np.array([0, 1, 0, 1, 1, 0, 1])
test_feat_vector2 = np.array([1, 0, 1, 0, 1, 1, 0])
test_feat_vector3 = np.array([1, 0, 1, 1, 0, 1, 1])
test_features = np.array([test_feat_vector1, test_feat_vector2, test_feat_vector3])
test_labels = np.array([4, 3, 6])

score, std = PADRE_predict(PADRE_model, test_features, train_features, train_labels)
print('Score:', score, '\n Std:', std)

Score: [3.60333333 3.52666667 3.59      ] 
 Std: [0.2868604  0.15173076 0.17146428]


### Testing on logP dataset

In [ ]:
PADRE_model_final = RandomForestRegressor()
PADRE_train(PADRE_model_final, dataset_final_train.X, dataset_final_train.y)

RandomForestRegressor()

In [ ]:
score_final, std_final = PADRE_predict(PADRE_model_final, dataset_final_test.X, dataset_final_train.X, dataset_final_train.y)
trues = dataset_final_test.y

print('preds', '\t', 'trues')
diffs = []
for i, j in zip(score_final, trues):
    print(round(i, 1), '\t', round(j, 1))
    diff = abs(i-j)
    diffs.append(diff)
print('Avg. diff:', np.mean(diffs))

preds 	 trues
3.3 	 4.4
3.3 	 2.6
3.1 	 2.7
0.9 	 -1.1
3.5 	 4.5
3.3 	 2.6
1.0 	 0.1
3.7 	 1.9
3.7 	 4.6
Avg. diff: 1.0581838775308654


Notes: So it seems that the more molecules I include from Jans implementation the worse the model becomes at predicting
Avg diff at 10 vs 90 in the training set equals approx. 1 vs 2.1

With PADREs own implementation we have an avg diff. (error) of approx 1 for both 10 and 90 molecules in the training set.

Both times I tested on 10 molecules. I noticed this as well when implemeniting the algorithm in MDRMF. The models seems to becomes worse and worse.
Conclusion: I have to reimplement everything in MDRMF.

### Testing using a larger dataset

In [ ]:
dataset = Dataset.load('datasets/dataset_morgan.pkl')
dataset_morgan_train = dataset.get_samples(150)

PADRE_model_morgan = RandomForestRegressor()
PADRE_train(PADRE_model_morgan, dataset_morgan_train.X, dataset_morgan_train.y)

RandomForestRegressor()

In [ ]:
dataset_morgan_test = dataset.get_samples(100)
score_morgan, std_morgan = PADRE_predict(PADRE_model_morgan, dataset_morgan_test.X, dataset_morgan_train.X, dataset_morgan_train.y)
trues = dataset_morgan_test.y

print('preds', '\t', 'trues')
diffs = []
for i, j in zip(score_morgan, trues):
    print(round(i, 1), '\t', round(j, 1))
    diff = abs(i-j)
    diffs.append(diff)
print('Avg. diff:', np.mean(diffs))

preds 	 trues
-7.9 	 -7.6
-8.9 	 -8.9
-8.0 	 -7.4
-7.6 	 -7.5
-8.2 	 -7.1
-8.8 	 -10.3
-8.3 	 -7.3
-8.0 	 -5.9
-7.4 	 -8.0
-7.8 	 -6.1
-6.8 	 -9.5
-8.1 	 -9.3
-6.5 	 -7.7
-8.3 	 -7.0
-8.6 	 -8.1
-8.3 	 -8.2
-8.5 	 -8.0
-8.1 	 -9.5
-7.4 	 -6.7
-8.1 	 -9.1
-8.4 	 -5.2
-8.1 	 -9.2
-7.7 	 -6.8
-7.9 	 -7.0
-7.4 	 -7.0
-7.5 	 -6.1
-6.9 	 -7.3
-7.0 	 -8.4
-6.8 	 -8.4
-6.4 	 -8.6
-6.3 	 -9.1
-8.5 	 -7.9
-7.2 	 -4.5
-7.7 	 -7.4
-7.9 	 -5.7
-6.8 	 -6.7
-9.2 	 -8.4
-8.2 	 -7.9
-7.5 	 -9.6
-8.4 	 -8.6
-5.8 	 -6.7
-8.4 	 -8.4
-8.5 	 -8.1
-8.1 	 -6.2
-7.9 	 -8.0
-10.0 	 -10.3
-7.7 	 -9.7
-7.6 	 -8.8
-8.1 	 -6.4
-8.5 	 -8.0
-8.1 	 -5.9
-9.0 	 -11.4
-8.7 	 -9.4
-8.4 	 -9.7
-8.2 	 -7.5
-8.9 	 -8.9
-8.3 	 -8.4
-8.3 	 -8.1
-8.1 	 -8.4
-8.7 	 -8.2
-8.3 	 -8.5
-8.2 	 -7.9
-6.4 	 -6.1
-8.2 	 -3.8
-8.2 	 -8.9
-8.0 	 -6.7
-8.1 	 -9.5
-8.8 	 -8.4
-8.7 	 -8.6
-8.7 	 -7.3
-8.2 	 -8.3
-8.7 	 -5.8
-6.6 	 -6.5
-8.1 	 -7.1
-7.0 	 -8.4
-7.5 	 -8.2
-8.2 	 -7.4
-7.8 	 -9.0
-8.1 	 -8.6
-8.4 	 -8.0
-7.6 	 -9.9
-8.7 	 -8.